In [2]:
from sklearn.cluster import KMeans
import numpy as np
X = [[1,1],[1.5,2],[3,4],[5,7],[3.5,5],[4.5,5],[3.5,4.5]]
print(np.array(X).shape)
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
print(kmeans.labels_)
print(kmeans.cluster_centers_)

(7, 2)
[1 1 0 0 0 0 0]
[[3.9  5.1 ]
 [1.25 1.5 ]]


In [34]:
import numpy as np
from numpy.linalg import norm 
import matplotlib.pyplot as plt
class Kmeans():
    """Kmeans _summary_
    """
    def __init__(self,X,no_of_clusters: int,init_centeroids=None): # X has number of rows equal obervation, number of columns equals dimnesion X.shape=[obervation,dimension]
        """__init__ _summary_

        Parameters
        ----------
        X : list[list[float]]
            design matrix
        no_of_clusters : int
            the number of desired clusters
        init_centeroids : list[list[float]], optional
            _description_, by default None
        """
        np.random.seed(0)
        self.X=np.array(X)
        self.cluster_labels=None
        self.no_of_clusters=no_of_clusters
        self.no_of_dimensions=self.X.shape[1]
        if init_centeroids==None:
            self.init_centroids=self.init_clusters()
        else:
            assert np.array(init_centeroids).shape==(self.no_of_clusters,self.no_of_dimensions), "Given centroids do not match shape"
            self.init_centroids=init_centeroids
        
    def transform(self):
        cluster_list=[]
        for i in range(len(self.init_centroids)):
            X=norm(self.X-self.init_centroids[i],2,axis=1).reshape(-1,1)
            cluster_list.append(X)
        self.cluster_labels=np.argmin(np.array(cluster_list),axis=0)
    def init_clusters(self):
        dump=[]
        X_min=np.min(self.X,axis=1)
        X_max=np.max(self.X,axis=1)
        for i in range(self.no_of_dimensions):
            dump.append(np.random.uniform(size=(self.no_of_clusters),low=X_min[i],high=X_max[i]))
        return np.array(dump).T
    def evaluate(self): # evaluate the choice of clusters number using sum of squared distance for each cluster
        return None
    def get_centroids(self):
        return self.init_centroids
    def get_cluster_labels(self):
        return self.cluster_labels
s=Kmeans([[1,2],[1,3],[1,7],[2,5],[5,6]],3,[[1,2.5],[5,5],[6,6]])
s.transform()
print(s.get_cluster_labels())

[[0]
 [0]
 [1]
 [0]
 [1]]
